In [1]:
import sys
from torch.utils.data import DataLoader
import torch
sys.path.append('../src')
from modules import (
                    paths,
                    dataset,
                    model
                    )
from torchvision.transforms import v2

/home/lexyo/Dev/cv-proj2/notebooks/../src/modules/paths.py


/home/lexyo/Dev/cv-proj2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
toy = True
batch_size = 2

In [3]:
import importlib
importlib.reload(dataset)
if toy == True:
    print("laoding toy datasets")
    train_dataset = dataset.load("train", tiny=True, stop=2)
    val_dataset = dataset.load("valid", tiny=True, stop=2)

else:
    print("loading full dataet")
    train_dataset = dataset.load("train", stop=2)
    val_dataset = dataset.load("valid", stop=2)
print("train:\n"+str(train_dataset))
print("validation:\n"+str(val_dataset))


laoding toy datasets
Loading dataset from /home/lexyo/Dev/cv-proj2/notebooks/../data/train.pkl
Loading dataset from /home/lexyo/Dev/cv-proj2/notebooks/../data/valid.pkl
train:
Dataset({
    features: ['image', 'label'],
    num_rows: 400
})
validation:
Dataset({
    features: ['image', 'label'],
    num_rows: 400
})


In [4]:
transform_train = v2.Compose([
    v2.Lambda(lambda x: x.convert('RGB')),  # some images are in grayscale
    v2.ToImage(), 
    v2.ToDtype(torch.float32, scale=True),
    v2.RandomHorizontalFlip(),
    v2.RandAugment(),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    v2.RandomErasing(p=0.25),

])

transform_valid = v2.Compose([
    v2.Lambda(lambda x: x.convert('RGB')),  # some images are in grayscale
    v2.ToImage(), 
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

In [5]:
train_loader = DataLoader(
    dataset.TorchDatasetWrapper(train_dataset, transform_train),
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,  
    pin_memory=True,
    prefetch_factor=4,
    persistent_workers=True
)

val_loader = DataLoader(
    dataset.TorchDatasetWrapper(val_dataset, transform_valid),
    batch_size=3,
    shuffle=False,
    num_workers=2,  
    pin_memory=True,
    prefetch_factor=4,
    persistent_workers=True
)

In [6]:
batch = next(iter(train_loader))

/home/lexyo/Dev/cv-proj2/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [37]:
importlib.reload(model)
config = {
    "patch_size": 8,
    "hidden_size": 48,
    "num_hidden_layers": 4,
    "num_attention_heads": 4,
    "intermediate_size": 4 * 48,
    "hidden_dropout_prob": 0.0,
    "attention_probs_dropout_prob": 0.0,
    "initializer_range": 0.02,
    "image_size": 32,
    "num_classes": 10,
    "num_channels": 3,
    "qkv_bias": True,
}
# embedding = model.Embeddings(config)
# x = embedding(batch[0])
# single = model.AttentionHead(1000, 10, 0.1)
# single(x)
# multi = model.MultiHeadAttention(config)
# multi(x, output_attentions = True)
# encoder = model.Encoder(config)
# encoder(x)
vit = model.ViT(config)
vit(batch[0], False)

KeyError: 'initializer_range'